In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [8]:
print("Reading data")
data = pd.read_csv("data/data2D.txt", sep=" ", header=None)

data

Reading data


,0,1
0,0.6803,-0.0163
1,3.8095,0.7985
2,-1.6614,-0.5777
3,-0.2573,-0.1556
4,0.6980,0.1746
...,...,...
1495,0.7957,0.2181
1496,1.2801,1.0967
1497,1.1920,2.2475
1498,0.4555,0.8846


In [ ]:
# Take a data file as input. The data file contains n data points, each having m attributes.
# As the number of components (or, the number of gaussian distributions, k) is usually unknown, you will assume a range for k. For example, from 1 to 10.
# For each value of k,
# Apply the EM algorithm to estimate the GMM.
# Keep a note of the converged log-likelihood.


In [ ]:
class GaussianMixtureModel:

    def __init__(self, n_components, max_iter=100, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        self.means = None
        self.covariances = None
        self.weights = None
    
    def fit(self, X):
        n_samples, n_features = X.shape
        
        # Initialize means, covariances, and weights
        self.means = np.random.rand(self.n_components, n_features)
        self.covariances = np.array([np.eye(n_features) for _ in range(self.n_components)])
        self.weights = np.ones(self.n_components) / self.n_components
        
        for i in range(self.max_iter):
            # Expectation step
            responsibilities = self._e_step(X)
            
            # Maximization step
            self._m_step(X, responsibilities)
            
            # Check for convergence
            if np.abs(self.weights.sum() - 1) < self.tol:
                break
    
    def predict(self, X):
        responsibilities = self._e_step(X)
        return responsibilities.argmax(axis=1)
    
    def _e_step(self, X):
        n_samples, _ = X.shape
        responsibilities = np.zeros((n_samples, self.n_components))
        for k in range(self.n_components):
            responsibilities[:, k] = self.weights[k] * self._multivariate_gaussian(X, self.means[k], self.covariances[k])
        responsibilities /= responsibilities.sum(axis=1, keepdims=True)
        return responsibilities
    
    def _m_step(self, X, responsibilities):
        n_samples, n_features = X.shape
        for k in range(self.n_components):
            # Update weights
            self.weights[k] = responsibilities[:, k].sum() / n_samples
            
            # Update means
            weighted_sum = responsibilities[:, k].dot(X)
            self.means[k] = weighted_sum / responsibilities[:, k].sum()
            
            # Update covariances
            centered_data = X - self.means[k]
            weighted_cov = np.dot(centered_data.T, responsibilities[:, k] * centered_data)
            self.covariances[k] = weighted_cov / responsibilities[:, k].sum()
    
    def _multivariate_gaussian(self, X, mean, covariance):
        n_samples = X.shape[0]
        X = X - mean
        return (2 * np.pi) ** (- X.shape[1] / 2) * np.linalg.det(covariance) ** -0.5 * np.exp(-0.5 * np.sum(X @ np.linalg.inv(covariance) * X, axis=1))